In [1]:
import sys
import os
sys.path.append(os.path.abspath('../'))

In [2]:
import pandas as pd

from mcda.electre.concordance import (
    concordance_with_interactions,
    concordance_with_interactions_marginal,
    FunctionType,
    Interaction,
    InteractionType
)
from mcda.core.scales import QuantitativeScale, PreferenceDirection
from mcda.core.functions import Threshold

1. Create dataframes

In [3]:
alternative_names = ['fiat', 'vwPolo' , 'nissan' , 'toyota' , 'suzuki', 'ford' ]
criteria_names = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6']
profile_names = ['P1', 'P2', 'P3']

In [4]:
cars = pd.DataFrame(
    [
        [4000, 311, 0.988, 90, 2, 8.5],
        [10000, 150, 0.8, 60, 4, 6.66],
        [21370, 255, 0.5, 90, 5, 7.77],
        [9999, 99, 0.9, 99, 9, 9.99],
        [12345, 12, 0.76, 43, 4, 1.5],
        [0, 1, 0, 0, 0, 0],
    ],
    index=alternative_names,
    columns=criteria_names,
)
cars

,C1,C2,C3,C4,C5,C6
fiat,4000,311,0.988,90,2,8.50
vwPolo,10000,150,0.800,60,4,6.66
nissan,21370,255,0.500,90,5,7.77
toyota,9999,99,0.900,99,9,9.99
suzuki,12345,12,0.760,43,4,1.50
ford,0,1,0.000,0,0,0.00


In [5]:
profiles = pd.DataFrame(
    [
        [1200, 25, 0.3, 40, 9, 7.0],
        [3500, 45, 0.6, 60, 4, 6.7],
        [3400, 90, 0.7654, 98, 2, 8.8]
    ],
    index=profile_names,
    columns=criteria_names
) 
profiles

,C1,C2,C3,C4,C5,C6
P1,1200,25,0.3000,40,9,7.0
P2,3500,45,0.6000,60,4,6.7
P3,3400,90,0.7654,98,2,8.8


2. Create series with thresholds and specify criteria scales and weights

In [6]:
scales = pd.Series([
    QuantitativeScale(0, 50000),
    QuantitativeScale(0, 1000),
    QuantitativeScale(0.0, 1.0),
    QuantitativeScale(0.0, 150.0),
    QuantitativeScale(0.0, 15.0, PreferenceDirection.MIN),
    QuantitativeScale(0.0, 15.0, PreferenceDirection.MIN),
], index=criteria_names)
scales

C1      Scale [0, 50000], max direction
C2       Scale [0, 1000], max direction
C3      Scale [0.0, 1.0], max direction
C4    Scale [0.0, 150.0], max direction
C5     Scale [0.0, 15.0], min direction
C6     Scale [0.0, 15.0], min direction
dtype: object

In [7]:
weights = pd.Series([2, 3, 5, 1, 3, 4], index=criteria_names)
weights

C1    2
C2    3
C3    5
C4    1
C5    3
C6    4
dtype: int64

In [8]:
indifference = pd.Series([
    Threshold(0.1, 1000),
    Threshold(0, 500),
    Threshold(0, 0.1),
    Threshold(0.1, 0),
    Threshold(0.2, 2),
    Threshold(0, 1.5),
], index=criteria_names)
indifference

C1    Threshold: alpha=0.1, beta=1000
C2       Threshold: alpha=0, beta=500
C3       Threshold: alpha=0, beta=0.1
C4       Threshold: alpha=0.1, beta=0
C5       Threshold: alpha=0.2, beta=2
C6       Threshold: alpha=0, beta=1.5
dtype: object

In [9]:
preference = pd.Series([
    Threshold(0.15, 1500),
    Threshold(0, 550),
    Threshold(0, 0.4),
    Threshold(0.4, 0.2),
    Threshold(0.2, 10),
    Threshold(0, 4.5),
], index=criteria_names)
preference

C1    Threshold: alpha=0.15, beta=1500
C2        Threshold: alpha=0, beta=550
C3        Threshold: alpha=0, beta=0.4
C4      Threshold: alpha=0.4, beta=0.2
C5       Threshold: alpha=0.2, beta=10
C6        Threshold: alpha=0, beta=4.5
dtype: object

In [10]:
interactions = pd.DataFrame([
    [None, None, None, None, None, None],
    [None, None, None, None, None, None],
    [Interaction(InteractionType.A, 3.4), None, None, None, None, None],
    [None, None, None, None, None, None],
    [None, None, None, None, None, Interaction(InteractionType.MS, 3.5)],
    [None, Interaction(InteractionType.MW, -4.6), None, None, None, None],
], index=criteria_names, columns=criteria_names)
interactions

,C1,C2,C3,C4,C5,C6
C1,None,None,None,None,None,None
C2,None,None,None,None,None,None
C3,Antagonistic | 3.4,None,None,None,None,None
C4,None,None,None,None,None,None
C5,None,None,None,None,None,Mutual Strengthening | 3.5
C6,None,Mutual Weakening | -4.6,None,None,None,None


3. Short example with comprehensive index calculations

Between fiat and suzuki:

In [11]:
concordance_with_interactions_marginal(
    cars.loc['fiat'],
    cars.loc['suzuki'],
    scales,
    weights,
    indifference,
    preference,
    interactions
)

0.589041095890411

5. Concordance table between alternatives and alternatives, with min function to capture the interactions effect

In [12]:
concordance_with_interactions(
    cars,
    scales,
    weights,
    indifference,
    preference,
    interactions,
)

,fiat,vwPolo,nissan,toyota,suzuki,ford
fiat,1.000000,0.819934,0.851852,0.851852,0.589041,0.777778
vwPolo,0.854043,1.000000,0.777778,0.925926,0.678767,0.752778
nissan,0.704142,0.802761,1.000000,0.704142,0.629630,0.736111
toyota,0.922581,0.796062,0.779497,1.000000,0.647792,0.669444
suzuki,0.814596,0.933929,0.777778,0.886442,1.000000,0.972519
ford,0.526627,0.526627,0.526627,0.526627,0.526627,1.000000


6. Concordance table between alternatives and profiles, with multiply function to capture the interactions effect.

In [13]:
alt_prof, prof_alt = concordance_with_interactions(
    cars,
    scales,
    weights,
    indifference,
    preference,
    interactions,
    FunctionType.MUL,
    profiles,
)

In [14]:
alt_prof

,P1,P2,P3
fiat,1.000000,0.970610,1.000000
vwPolo,1.000000,1.000000,0.940828
nissan,1.000000,1.000000,0.836884
toyota,0.886127,0.836452,0.922581
suzuki,1.000000,0.942635,0.940828
ford,0.674419,0.526627,0.526627


In [15]:
prof_alt

,fiat,vwPolo,nissan,toyota,suzuki,ford
P1,0.406452,0.483969,0.674019,0.526627,0.363889,0.627193
P2,0.654021,0.749431,0.777778,0.598309,0.541885,0.752778
P3,0.862767,0.792253,0.851852,0.814523,0.589041,0.777778
